**Outline**
- [Implementing RNNs for sequence modeling in PyTorch](#Implementing-RNNs-for-sequence-modeling-in-PyTorch)
  - [Predicting the sentiment of IMDb movie reviews](#Predicting-the-sentiment-of-IMDb-movie-reviews)
  - [Preparing the movie review data](#Preparing-the-movie-review-data)
  - [Embedding layers for sentence encoding](#Embedding-layers-for-sentence-encoding)
  - [Building an RNN model](#Building-an-RNN-model)
  - [Building an RNN model for the sentiment analysis task](#Building-an-RNN-model-for-the-sentiment-analysis-task)
  - [The bidirectional RNN](#The-bidirectional-RNN)

# Implementing RNNs for sequence modeling in PyTorch

## Predicting the sentiment of IMDb movie reviews

The sentiment analysis (language modeling) is concerned with analyzing the expressed opinion of a sentence or a text document. This can be very useful in creating a sentiment score for an exchange-listed firm or even a financial index by creating a sentiment score based on news articles and analyst reports. This is typically an approach which is used to automate the economicy policy sentiment indices such as the one developed by Baker and Bloom: https://www.policyuncertainty.com/

As any sentiment score would be based on a long sequence of text inputs, we implement a many-to-many RNN.

### Preparing the movie review data

In [ ]:
from IPython.display import Image
%matplotlib inline

Each set has 25,000 samples. And each sample of the datasets consists of two elements, the sentiment label representing the target label we want to predict (`neg` refers to negative sentiment and `pos` refers to positive sentiment), and the movie review text (the input features). The text component of these movie reviews is sequences of words, and the RNN model classifies each sequence as a positive (`1`) or negative (`0`) review.

Before we feed the data into an RNN model, we need to apply several preprocessing steps:
1. Split the training dataset into separate training and validation partitions.
2. Identify the unique words in the training dataset
3. Map each unique word to a unique integer and encode the review text into encoded integers
(an index of each unique word)
4. Divide the dataset into mini-batches as input to the model

First, we import the necessary modules and read the data from `torchtext` (which we install via `pip install torchtext`). To import data, we need to install `torchdata` as well.

In [ ]:
!pip uninstall torch

In [ ]:
!pip install --pre torch -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

In [ ]:
!pip install --user "git+https://github.com/pytorch/data.git"

In [ ]:
!pip install -U torchtext==0.10.0

At this point, in the menu, go to **Runtime** and **Restart Runtime**.

In [ ]:
from torchtext.datasets import IMDB
from torch.utils.data.dataset import random_split

# Step 1: load and create the datasets

train_dataset = IMDB(split='train')
test_dataset = IMDB(split='test')

In [ ]:
import torch
torch.manual_seed(1)
train_dataset, valid_dataset = random_split(list(train_dataset), [20000, 5000])


The original training dataset contains 25,000 examples. 20,000 examples are randomly chosen for training, and 5,000 for validation.

To prepare the data for input to an NN, we need to encode it into numeric values. To do this, we first find the unique words (tokens) in the training dataset. It is more efficient to use the `Counter` class from the `collections` package, which is part of Python’s standard library.

In the following code, we instantiate a new `Counter` object (`token_counts`) that collects the unique word frequencies. Note that in this particular application (and in contrast to the `bag-of-words` model), we are only interested in the set of unique words and won’t require the word counts, which are created as a side product. In the `bag-of-words` model, we would have required to keep track of word count as well.

The `tokenizer` function removes HTML markups as well as punctuation and other non-letter characters.

In [ ]:
## Step 2: find unique tokens (words)
import re
from collections import Counter, OrderedDict

token_counts = Counter()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized


for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)
 
    
print('Vocab-size:', len(token_counts))

We map each unique word to a unique integer. This can be done manually using a Python dictionary, where the keys are the unique tokens (words) and the value associated with each key is a unique integer. However, the `torchtext` package already provides a class, `Vocab`, which we use to create such a mapping and encode the entire dataset. 

First, we create a `vocab` object by passing the ordered dictionary mapping tokens to their corresponding occurrence frequencies
(the ordered dictionary is the sorted `token_counts`). Second, we prepend two special tokens to the vocabulary – the padding and the unknown token.

To demonstrate how to use the `vocab` object, we convert an example input text into a list of integer values.

In [ ]:
## Step 3: encoding each unique token into integers
from torchtext.vocab import vocab

sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)

vocab = vocab(ordered_dict)

vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)

print([vocab[token] for token in ['bad', 'movie', 'terrible', 'watch']])

Note that there might be some tokens in the validation or testing data that are not present in the training
data and are thus not included in the mapping. If we have $q$ tokens (that is, the size of `token_counts`
passed to `Vocab`, which in this case is 69,023), then all tokens that haven’t been seen before, and are
thus not included in `token_counts`, will be assigned the integer `1` (a placeholder for the unknown token).
In other words, the index `1` is reserved for unknown words. Another reserved value is the integer
`0`, which serves as a placeholder, a so-called padding token, for adjusting the sequence length. This is handy when we are building an RNN model in PyTorch.

Next we define the `text_pipeline` function to transform each text in the dataset accordingly and the `label_pipeline` function to convert each label to 1 or 0.

In [ ]:
## Step 3-A: define the functions for transformation

# device = torch.device("cuda:0") # use this when GPU on the computing device
device = 'cpu'

text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: 1. if x == 'pos' else 0.

We generate batches of samples using `DataLoader` and pass the data processing pipelines declared previously to the argument `collate_fn`. We wrap the text encoding and label transformation function into the `collate_batch` function.

In [ ]:
## Step 3-B: wrap the encode and transformation function
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), 
                                      dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True)
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

Above we convert sequences of words into sequences of integers, and labels of `pos` or `neg` into 1 or 0. However, the sequences currently have different lengths (as shown in the result of executing the following code for four examples). Although, in general, RNNs can handle sequences with different lengths, we still need to make sure that all the sequences in a mini-batch have the same length to store them efficiently in a tensor.

PyTorch provides an efficient method, `pad_sequence()`, which automatically pads the consecutive elements that are to be combined into a batch with placeholder values (0s) so that all sequences within a batch will have the same shape. 

In the previous code, we already created a data loader of a small batch size from the training dataset and applied the `collate_batch` function, which itself included a `pad_sequence()` call.

In [ ]:
## Take a small batch
import torch.nn as nn

from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, collate_fn=collate_batch)
text_batch, label_batch, length_batch = next(iter(dataloader))
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)

As we can see above, after the padding has been applied, the batch has all 4 examples of length 218.

Finally, we divide all three datasets into data loaders with a batch size of 32.

In [ ]:
## Step 4: batching the datasets

batch_size = 32  

train_dl = DataLoader(train_dataset, batch_size=batch_size,
                      shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size,
                      shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_dataset, batch_size=batch_size,
                     shuffle=False, collate_fn=collate_batch)

Now, the data is in a suitable format for an RNN model.

We first discuss feature **embedding**, which is an optional but highly recommended preprocessing step that is used to reduce the dimensionality of the word vectors. Otherwise the input dimension could be unnecessarily large (think of dimension being the length of the input examples vocabulary), which can slow down the training or even result in a bad model fit.

### Embedding layers for sentence encoding

During the data preparation in the previous step, we generated sequences of the same length. The elements of these sequences were integer numbers that corresponded to the indices of unique words. These word indices can be converted into input features in several different ways. 

One naive way is to apply one-hot encoding to convert the indices into vectors of zeros and ones. Then, each word will
be mapped to a vector whose size is the number of unique words in the entire dataset. Given that the number of unique words (the size of the vocabulary) can be in the order of $10^4$ – $10^5$, which will also be the number of our input features, a model trained on such features may suffer from the **curse of dimensionality**. Furthermore, these features are very sparse since all are zero except one.

A more elegant approach is to map each word to a vector of a fixed size with real-valued elements (not necessarily integers). In contrast to the one-hot encoded vectors, we can use finite-sized vectors to represent an infinite number of real numbers. (In theory, we can extract infinite real numbers from a given interval, for example [–1, 1].)

This is the idea behind embedding, which is a feature-learning technique that we utilize here to automatically learn the salient features to represent the words in our dataset.

Given the number of unique words, $n_{words}$, we can select the size of the embedding vectors (a.k.a., embedding dimension)
to be much smaller than the number of unique words ($embedding\_dim << n_{words}$) to represent the entire vocabulary as input features.

The advantages of embedding over one-hot encoding are as follows:
- A reduction in the dimensionality of the feature space to decrease the effect of the curse of dimensionality
- The extraction of salient features since the embedding layer in an NN can be optimized (or learned)

Given a set of tokens of size $n_{words} + 2$ ($n_{words}$) is the size of the token set, plus index 0 is reserved for the padding placeholder, and 1 is for the words not present in the token set), an embedding matrix of size
$(n_{words} + 2) \times embedding\_dim$ will be created where each row of this matrix represents numeric features
associated with a token. Therefore, when an integer index, $i$, is given as input to the embedding, it
will look up the corresponding row of the matrix at index $i$ and return the numeric features. 

The embedding matrix serves as the input layer to the NN models. In practice, creating an embedding layer can simply be done using `nn.Embedding`. Let’s see an example where we create an embedding layer and apply it to a batch of two samples.

In [ ]:
embedding = nn.Embedding(num_embeddings=10, 
                         embedding_dim=3, 
                         padding_idx=0)
 
# a batch of 2 samples of 5 indices each
text_encoded_input = torch.LongTensor([[1,2,4,5,6],[4,3,2,6,0]])
print(embedding(text_encoded_input))

The input to this model (embedding layer) must have rank 2 with the dimensionality $batchsize \times input\_length$, where $input\_length$ is the length of sequences (here, 5).

For example, an input sequence in the mini-batch could be $<1, 8, 5, 9, 2>$, where each element of this sequence is the index of the unique words. The output will have the dimensionality $batchsize \times input\_length \times embedding\_dim$, where $embedding\_dim$ is the size of the embedding features (here, set to 4).

The other argument provided to the embedding layer, `num_embeddings`, corresponds to the unique integer values that the model will receive as input (for instance, $n + 2$, set here to 10). Therefore, the embedding matrix in this case has the size $10 \times 4$.

`padding_idx` indicates the token index for padding (here, 0), which, if specified, will not contribute to the gradient updates during training. In our example, the length of the original sequence of the second sample is 4, and we padded it with 1 more element 0. The embedding output of the padded element is [0, 0, 0, 0].

### Building an RNN model

Now we are ready to build an RNN model. Using the `nn.Module` class, we can combine the embedding layer, the recurrent layers of the RNN, and the fully connected non-recurrent layers. For the recurrent layers, we can use any of the following implementations:


* **RNN layers:**
  * `nn.RNN(input_size, hidden_size, num_layers=1)`
  * `nn.LSTM(..)`
  * `nn.GRU(..)`
  * `nn.RNN(input_size, hidden_size, num_layers=1, bidirectional=True)`
 
 

To see how a multilayer RNN model can be built using one of these recurrent layers, in the following example, we create an RNN model with two recurrent layers of type RNN. Finally, we add a non-recurrent fully connected layer as the output layer, which returns a single output value as the prediction.

In [ ]:
## An example of building a RNN model
## with simple RNN layer

# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, 
                          hidden_size, 
                          num_layers=2, 
                          batch_first=True)
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1) # output dimension is 1
        
    def forward(self, x):
        _, hidden = self.rnn(x)
        out = hidden[-1, :, :] # take the output of last recurrent hidden layer
        out = self.fc(out)
        return out

model = RNN(64, 32) 

print(model) 
 
model(torch.randn(10, 3, 64)) # Recall the input shape is (batch_size, sequence_length, features)

### Building an RNN model for the sentiment analysis task

Since we have very long sequences in the movie reviews, we use an LSTM layer to account for long-range effects. We create an RNN model for sentiment analysis, starting with an embedding layer producing word embeddings of feature size 20 (`embed_dim=20`).

Then, a recurrent layer of type LSTM will be added. Finally, we add a fully connected layer as a hidden layer and another fully connected layer as the output layer, which returns a single class-membership probability value via the logistic sigmoid
activation as the prediction.

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)  # checks for the first dimension being the batch size
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out) # note the cell output from LSTM
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
         
vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)
model

Next we develop the `train` function to train the model on the given dataset for one epoch and return the classification accuracy and loss.

In [ ]:
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)
 
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)

Similarly, we will develop the `evaluate` function to measure the model’s performance on a given dataset. For a binary classification with a single class-membership probability output, we use the binary cross-entropy loss (`BCELoss`) as the loss function.

We train the model for 10 epochs and display the training and validation performances.

In [ ]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5

torch.manual_seed(1)
 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')
 

After training this model for 10 epochs, we evaluate it on the test data.

In [ ]:
acc_test, _ = evaluate(test_dl)
print(f'test_accuracy: {acc_test:.4f}') 

Note that the above model is not the best when compared to the state-of-the-art methods used on the IMDb dataset.

#### The bidirectional RNN

We can set the bidirectional configuration of the LSTM to `True`, which will make the recurrent layer pass through the input sequences from both directions, start to end, as well as in the reverse direction.

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(rnn_hidden_size*2, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        _, (hidden, cell) = self.rnn(out)
        out = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
    
torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)

The bidirectional RNN layer makes two passes over each input sequence: a forward pass and a reverse
or backward pass (note that this is not to be confused with the forward and backward passes in the
context of backpropagation). The resulting hidden states of these forward and backward passes are
usually concatenated into a single hidden state. Other merge modes include summation, multiplication
(multiplying the results of the two passes), and averaging (taking the average of the two).

In [ ]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

num_epochs = 5 

torch.manual_seed(1)
 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')

In [ ]:
test_dataset = IMDB(split='test')
test_dl = DataLoader(test_dataset, batch_size=batch_size,
                     shuffle=False, collate_fn=collate_batch)

In [ ]:
acc_test, _ = evaluate(test_dl)
print(f'test_accuracy: {acc_test:.4f}') 

We can also try other types of recurrent layers, such as the regular RNN. However, as it turns out, a model
built with regular recurrent layers won’t be able to reach a good predictive performance (even on the
training data). For example, if you try replacing the bidirectional LSTM layer in the previous code with
a unidirectional `nn.RNN` (instead of `nn.LSTM`) layer and train the model on full-length sequences, you
may observe that the loss will not even decrease during training. The reason is that the sequences in
this dataset are too long, so a model with an RNN layer cannot learn the long-term dependencies and
may suffer from vanishing or exploding gradient problems.